# Separate Stacking Model

Here, we train a meta-learner that will best combine the predictions from the sub-models and ideally perform better than any single sub-model. The three sub-models are:
 - 2 MODNet models trained on the HSE and experimental datasets
 - 1 pretrained MEGNet model for the PBE dataset.

In [1]:
def setup_threading():
    import os
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["TF_NUM_INTRAOP_THREADS"] = "1"
    os.environ["TF_NUM_INTEROP_THREADS"] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
setup_threading()

In [3]:
from modnet.preprocessing import MODData
from modnet.models.vanilla import MODNetModel
from modnet.hyper_opt.fit_genetic import FitGenetic
import numpy as np
import pandas as pd
import os
import copy

In [4]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

#### Loading the PBE, HSE and experimental datasets.

In [5]:
md_pbe = MODData.load('pbe_gap.zip')
md_pbe.df_targets.columns = ['gap']
md_hse = MODData.load('hse_gap.zip')
md_hse.df_targets.columns = ['gap']
md_exp = MODData.load('exp_gap_all')
md_exp.df_targets.columns = ['gap']
md_joint = MODData.load('exp_pbe_joint')


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766

2021-06-14 17:50:02,763 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f2cf37d43a0> object, created with modnet version <=0.1.7
2021-06-14 17:50:06,197 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f2cf37d4f40> object, created with modnet version 0.1.8~develop
2021-06-14 17:50:10,579 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f2cf37d49a0> object, created with modnet version <=0.1.7
2021-06-14 17:50:12,394 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f2cf37db6d0> object,

In [ ]:
md_joint.df_targets = md_joint.df_targets.drop(columns=['pbe_gap', 'difference'])

#### MEGNet model trained on the PBE dataset

In [6]:
import tensorflow as tf
from megnet.data.crystal import CrystalGraph
from megnet.data.graph import GaussianDistance
from megnet.models import MEGNetModel

#Pretrained model (we are using transfer learning)

model_form = MEGNetModel.from_file('band_gap_regression.hdf5')

#### MODNet model trained on the HSE dataset

In [8]:
model2 = MODNetModel.load('MODNetModel2')

2021-06-14 17:51:05,078 - modnet - INFO - Loaded <modnet.models.ensemble.EnsembleMODNetModel object at 0x7f2bfc463c40> object, created with modnet version 0.1.9


INFO:modnet:Loaded <modnet.models.ensemble.EnsembleMODNetModel object at 0x7f2bfc463c40> object, created with modnet version 0.1.9


### Separate Stacking Model

In [10]:
# stacked generalization with linear meta model on blobs dataset
import pandas as pd
from sklearn.metrics import accuracy_score

In [11]:
# load the saved models
def load_all_models():
    all_models = list()
    all_models = [model_form, model2, model3]
    return all_models

In [12]:
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX, inputX_meg):
    pred_meg = pd.DataFrame([model_form.predict_structure(x) for x in inputX_meg['structure']])
    gap = pred_meg[pred_meg.columns[0]]
    pred_meg = pred_meg.reindex(model2.predict(inputX).index)
    pred_meg['gap'] = gap
    pred_meg = pred_meg.drop(pred_meg.columns[0], axis=1)
    for i in range(len(pred_meg)):
        pred_meg.iloc[i] = gap[i]
    
    stackX = pd.concat([pred_meg, model2.predict(inputX), model3.predict(inputX)], axis=1).to_numpy()
    return stackX

In [13]:
# Optional cell if you want to customize the model used in the fit_stacked_model() function (instead of the LinearRegression())
import tensorflow as tf

def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(3, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam',
                loss='mse',
                metrics=['accuracy'])
    return model

In [14]:
# fit a model based on the outputs from the ensemble members
from sklearn.linear_model import LinearRegression

def fit_stacked_model(members, inputX, inputy, inputX_meg):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX, inputX_meg)
    # fit standalone model
    model = LinearRegression()
    model.fit(stackedX, inputy)
    return model

In [15]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX, inputX_meg):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX, inputX_meg)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [16]:
import math
from pymatgen.ext.matproj import MPRester
from pymatgen.core.structure import Structure

k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
             
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    fpath_meg = 'train_meg_{}_{}.pkl'.format(random_state,i+1)
    if os.path.exists(fpath_meg):
        train_meg = pd.read_pickle(fpath_meg)
    else:
        with MPRester("v454AlrxL5Zg0cLAXW0") as mpr:
            for a in range(len(train.df_structure)):
                try:
                    structure = mpr.get_doc(mpr.get_materials_id_from_task_id(train.df_structure.index[a]))['structure']
                    train.df_structure['structure'][a] = structure
                except:
                    train.df_structure['structure'][a] = math.nan      
        for b in range(len(train.df_structure)):
            structure = Structure.from_dict(train.df_structure['structure'][b])
            train.df_structure['structure'][b] = structure
        train_meg = train.df_structure
        train_meg.to_pickle(fpath_meg)
    
    fpath_meg_test = 'train_meg_test_{}_{}.pkl'.format(random_state,i+1)
    if os.path.exists(fpath_meg_test):
        test_meg = pd.read_pickle(fpath_meg_test)
    else:
        with MPRester("v454AlrxL5Zg0cLAXW0") as mpr:
            for c in range(len(test.df_structure)):
                try:
                    structure = mpr.get_doc(mpr.get_materials_id_from_task_id(test.df_structure.index[c]))['structure']
                    test.df_structure['structure'][c] = structure
                except:
                    test.df_structure['structure'][c] = math.nan
        for d in range(len(test.df_structure)):
            structure = Structure.from_dict(test.df_structure['structure'][d])
            test.df_structure['structure'][d] = structure
        test_meg = test.df_structure
        test_meg.to_pickle(fpath_meg_test)
    
    ga = FitGenetic(train)
    model3 = ga.run(size_pop=20, num_generations=10, n_jobs=10)
    
    # load all models
    members = [model_form, model2, model3]
    print('Loaded %d models' % len(members))
    
    # fit stacked model on test dataset
    model = fit_stacked_model(members, inputX=train, inputy=train.df_targets.to_numpy(), inputX_meg=train_meg)

    # evaluate model on test set
    pred = stacked_prediction(members, model, test, test_meg)
    pred = pd.DataFrame(pred)
    pred = pred.rename(columns={pred.columns[0]: 'exp_gap'})
    true = test.df_targets
    for j in range(len(pred)):
        pred = pred.rename(index={pred.index[j]: true.index[j]})
    error = pred-true
    error = error['exp_gap'].drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('MAE =')
    print(mae)
    maes[i] = mae

2021-06-14 17:51:05,303 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.


INFO:modnet:Loaded DeBreuck2020Featurizer featurizer.


2021-06-14 17:51:05,330 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.


INFO:modnet:Loaded DeBreuck2020Featurizer featurizer.


2021-06-14 17:51:05,349 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.


...

2021-06-15 00:46:05,844 - modnet - INFO - Loss per individual: ind 0: 0.665 	ind 1: 0.609 	ind 2: 0.590 	ind 3: 0.643 	ind 4: 0.676 	ind 5: 0.630 	ind 6: 0.630 	ind 7: 0.614 	ind 8: 0.649 	ind 9: 0.667 	ind 10: 0.716 	ind 11: 0.619 	ind 12: 0.643 	ind 13: 0.642 	ind 14: 0.655 	ind 15: 0.655 	ind 16: 0.604 	ind 17: 0.744 	ind 18: 0.635 	ind 19: 0.626 	



INFO:modnet:Loss per individual: ind 0: 0.665 	ind 1: 0.609 	ind 2: 0.590 	ind 3: 0.643 	ind 4: 0.676 	ind 5: 0.630 	ind 6: 0.630 	ind 7: 0.614 	ind 8: 0.649 	ind 9: 0.667 	ind 10: 0.716 	ind 11: 0.619 	ind 12: 0.643 	ind 13: 0.642 	ind 14: 0.655 	ind 15: 0.655 	ind 16: 0.604 	ind 17: 0.744 	ind 18: 0.635 	ind 19: 0.626 	


2021-06-15 00:46:07,809 - modnet - INFO - Early stopping: same best model for 4 consecutive generations


INFO:modnet:Early stopping: same best model for 4 consecutive generations


Loaded 3 models
MAE =
0.37428165677317615


In [17]:
maes

array([0.40554031, 0.35850184, 0.30848659, 0.34325991, 0.37428166])

In [18]:
maes.mean()

0.35801406179373074

### Conclusion

Relative percentage change of the error compared to MODNet_exp_GA reference: -3%